In [2]:
import getpass
import os

# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI

# GPT-4 모델로 챗봇 초기화
llm = ChatOpenAI(model="gpt-4-turbo")

# 3가지 프롬프트로 구분
1. 글구조 프롬프트
2. 글 생성 프롬프트
3. 글 평가 프롬프트


## 1.글 구조 프롬프트

In [2]:
#벤치마크 글 pdf 파일 -> str 
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader

pdf_filepath = "C:/Users/vjame/Downloads/__.Print (1).pdf"
loader = PyPDFLoader(pdf_filepath)
pages = loader.load()
page_benchmark=""
for i in range(len(pages)):
          page_benchmark+=pages[i].page_content[125:]
page_benchmark

"2월 22일  서울  지하철  폭설  연착  사태 : 봄의  기대와  겨울의  잔인함  사이주절주절59분 전\nhttps://blog.na ver.com/minhapp y68/223421938981\n지하철 온다 대표 마스코트 '오지통'\n안녕하세요 , 여러분의  출퇴근  메신저  지하철  온다의   '오.지 .통  [ 오늘의  지하철  소식통 ]' 인사 드립니다 !\n오늘  아침 , 서울은  갑작스런  폭설로  인해  큰  혼란을  겪었습니다 . \n많은  시민들이  불편을  호소하며  SNS 에  그들의  경험을  공유했습니다 .\n 특히  지하철  연착은  출근길을  더욱  어렵게  만들었습니다 .\n지연/사고 \n지연/사고  일시 : 2024년  2 월  22 일  오전\n지연/사고  노선 : 서울지하철  5 호선 , 2 호선 , 7 호선\n지연/사고  이유 : 강동구  고덕기지  지상구간에  쌓인  눈으로  인한  전력  공급  지장 , 승강장  안전문  장애\n문의 사항 : 서울교통공사  홈페이지\n이번  폭설은  서울  강동구  고덕기지의  지상구간에  밤새  쌓인  눈으로  인해  전력  공급에  지장을  주었고 ,\n 이로  인해  열차가  지연되었습니다 . \n서울지하철  5 호선은  첫차부터  25 분씩  지연되었으며 ,\n 2호선과  7 호선  또한  비슷한  연착을  겪었습니다 . \n이로  인한  출근길  혼잡은  시민들에게  큰  불편을  주었고 , \nSNS를  통해  다양한  불만이  제기되었습니다 .\n지하철 연착으로 어려운 출근길, 그래도 봄은 오고 \n있어요!\n이번  사건은  또  한  번  우리에게  자연의  힘  앞에  우리가  얼마나  작은  존재인지를  일깨워줍니다 . \n하지만  힘든  순간일수록  서로를  응원하고 , 공감하며  지내는  것이  중요합니다 . \n패딩을  다시  꺼내야  할  정도로  추운  날씨  속에서도 , \n우리  모두  봄을  기다리는  마음으로  따뜻하게  지내봅시다 !\n오지통이  

In [3]:
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "Extract the table of structure from the provided text in English. Please don't give me the details of the structure of the article JUST STRUCTURE 1: Greeting 2:"), #"목차정보 추출"
    ("user", "{input}")
])
chain= prompt1|llm
chain.invoke({"input":page_benchmark})

AIMessage(content='1: Greeting\n2: Introduction to the Mascot\n3: Description of the Incident\n4: Details of the Delay/Accident\n5: Encouragement and Conclusion\n6: Service Promotion', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 1240, 'total_tokens': 1282}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_3450ce39d5', 'finish_reason': 'stop', 'logprobs': None}, id='run-dea30080-74d1-4371-b471-fb7c00d66727-0')

In [5]:
"1. Greeting from 'Oji-Tong', the mascot of 'Subway Onda'\n2. Overview of the subway delay incident due to heavy snowfall in Seoul\n3. Details of the delay:\n   - Date and time of the incident\n   - Affected subway lines\n   - Reasons for the delay\n4. Impact of the delays on commuters and reactions on social media\n5. Encouragement and reminder of the inevitability of spring following winter\n6. Promotion of the 'Subway Onda' service and its features\n7. Closing encouragement and anticipation for better days"

1. Greeting from 'Oji-Tong', the mascot of 'Subway Onda'
2. Overview of the subway delay incident due to heavy snowfall in Seoul
3. Details of the delay:
   - Date and time of the incident
   - Affected subway lines
   - Reasons for the delay
4. Impact of the delays on commuters and reactions on social media
5. Encouragement and reminder of the inevitability of spring following winter
6. Promotion of the 'Subway Onda' service and its features
7. Closing encouragement and anticipation for better days


In [3]:
#글 구조 리스트로 변형 (task 쪼개기 위함)
import re
#글의 구조에서 너무 자세한 정보는 직접 제거하였음.
output1 = "1. Greeting from 'Oji-Tong', the mascot of 'Subway Onda'\n2. Overview of the subway delay incident \n3. Details of the delay:\n   - Date and time of the Delay\n   - Affected subway lines\n   - Reasons for the delay\n4. Encouragement and reminder Delay or cause of Delay \n6. Promotion of the 'Subway Onda' service and its features\n7. Closing encouragement and anticipation for better days"

# Using regex to split the string into sections starting with number and period
sections = re.split(r'\n(?=\d+\.)', output1)
section=""
for i in (sections):
          section+=i

In [4]:
section

"1. Greeting from 'Oji-Tong', the mascot of 'Subway Onda'2. Overview of the subway delay incident 3. Details of the delay:\n   - Date and time of the Delay\n   - Affected subway lines\n   - Reasons for the delay4. Encouragement and reminder Delay or cause of Delay 6. Promotion of the 'Subway Onda' service and its features7. Closing encouragement and anticipation for better days"

## 2. 글 생성 프롬프트

추후에 Injection이 완료되면 source_link가 아닌 text 혹은 요약된 text로 전달받을 예정

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
#템플릿 생성 - 채팅 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
# 뉴스기사 링크 (source data)
source_link=["https://www.news1.kr/articles/5152573","https://www.joongboo.com/news/articleView.html?idxno=363604327","https://www.kyeonggi.com/article/20230215580187","https://www.gukjenews.com/news/articleView.html?idxno=2969525"]

In [24]:
# 글 생성 프롬프트
prompt = ChatPromptTemplate(
messages=[
SystemMessagePromptTemplate.from_template(
          """The system utilizes the structure from existing blog benchmark articles and creates new blog posts using information from new source data.
          structure from existing blog benchmark articles : """+section+"""
          Once you obtain information from the source data, you will write the article considering the structure from existing blog benchmark articles.
          """
),
# The `variable_name` here is what must align with memory
MessagesPlaceholder(variable_name="chat_history"),
HumanMessagePromptTemplate.from_template("{question}")
]
)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
conversation = LLMChain(
llm=llm,
prompt=prompt,
verbose=True,
memory=memory
)

conversation({"question":'source data:{'+source_link[0] +'}Do not start writing until you are given a command. Just Review source data'})
content=""
for j in sections:
          content+=conversation.predict(question=j)
#시스템프롬프트수정 - 번역 & 대학생서포터즈
conversation.predict(question="Translate English to Korean")
result=conversation.predict(question="대학생서포터즈가 작성한 것 처럼 통통 튀게 작성해줘.")




> Entering new LLMChain chain...
Prompt after formatting:
System: The system utilizes the structure from existing blog benchmark articles and creates new blog posts using information from new source data.
          structure from existing blog benchmark articles : 1. Greeting from 'Oji-Tong', the mascot of 'Subway Onda'2. Overview of the subway delay incident 3. Details of the delay:
   - Date and time of the Delay
   - Affected subway lines
   - Reasons for the delay4. Encouragement and reminder Delay or cause of Delay 6. Promotion of the 'Subway Onda' service and its features7. Closing encouragement and anticipation for better days
          Once you obtain information from the source data, you will write the article considering the structure from existing blog benchmark articles.
          
Human: source data:{https://www.news1.kr/articles/5152573}Do not start writing until you are given a command. Just Review source data

> Finished chain.


> Entering new LLMChain chain...
Promp

In [26]:
result

["안녕하세요, 여러분! 저희 '지하철 온다'의 마스코트 오지통이 인사드립니다. 오늘도 여러분의 출퇴근길이 순조로우셨기를 바라며, 최근 있었던 지하철 지연 사고에 대해 알려드리려고 해요.\n\n**지연 사고 개요:**\n우리 모두가 불편을 겪었던 그 사고! 바로 어제, 출근 시간대에 발생한 지하철 지연 사건이랍니다.\n\n**세부사항:**\n- **일시**: 2023년 10월 5일 오전 8시\n- **영향을 받은 노선**: 2호선 및 7호선\n- **지연 원인**: 신호 장애로 인해 발생했어요.\n\n이런 사고가 발생하면 정말 답답하죠? 하지만 이런 사고를 통해 우리는 더 안전하고 믿을 수 있는 서비스를 제공하기 위한 중요한 교훈을 얻게 되어요. 잠시의 불편함을 겪을 수는 있지만, 이를 통해 더 나은 서비스를 위한 발판을 마련하고 있다고 생각해 주세요.\n\n**지하철 온다 서비스 홍보:**\n'지하철 온다' 앱을 통해 실시간 업데이트를 받아보세요. 또한, 경로를 계획할 수 있는 사용자 친화적인 기능들을 많이 갖추고 있답니다. 여러분의 피드백은 우리 서비스 개선에 큰 도움이 됩니다. 자주 이용하시는 분들을 위한 로열티 프로그램도 잊지 마세요!\n\n**마무리 인사:**\n앞으로 더 밝고 매끄러운 날들이 기다리고 있다고 확신합니다. '지하철 온다'는 계속해서 여러분의 편안하고 신속한 여행을 위해 노력할 것입니다. 이번 지연 사건에 대한 여러분의 인내와 이해에 감사드리며, 앞으로 더욱 편리하고 지연 없는 여행을 기대해 주세요. 오늘도 안전하고 즐거운 하루 되세요!"]

## 3.글 평가 프롬프트

### 3-1 평가 방법 

In [12]:
eval=["""Compare and evaluate the blog content you created with the article.
First, Extract the key information from the article. (time,date,subway line)
You need to decide whether the key information is entailed by the  CONTEXT by choosing one of the following rating: 

1. 5 point: The blog content follows logically from the information contained in the article. 

2. 1 point: The blog content is logically false from the information contained in the article. 

3. an integer score between 1 and 5 and if such integer score does not exist,  

use 1: It is not possible to determine whether the blog content is true or false without further information. 

Read the passage of information thoroughly and select the correct answer from the three answer labels. 

Read the CONTEXT thoroughly to ensure you know what the CONTEXT entails.  

Note the blog content is generated by a computer system, it can contain certain symbols, which should not be a negative factor in the evaluation.


Evaluation Form (scores ONLY):

- Consistency:
          
          """,
"""Your task is to rate the blog content on one metric.

          Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

          Evaluation Criteria:

          Coherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby "the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic."

          Evaluation Steps:

          1. Read the news article carefully and identify the main topic and key points.
          2. Read the blog content and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it presents them in a clear and logical order.
          3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.



          Evaluation Form (scores ONLY):

          - Coherence:
          
          """,
"""Friendliness, as a metric, measures how effectively the text of a blog content engages the reader with a friendly and inviting tone. If the tone and style of the text foster a sense of warmth and familiarity, making the reader feel welcomed and engaged, then the value of the Friendliness metric should be high; otherwise, it should be low. After reading the blog post, evaluate its tone and style against typical friendly communication, and determine the value of the Friendliness metric using the following rating scale:

          - One star: The tone is cold or unfriendly, not engaging the reader in any appreciable way.
          - Two stars: The tone has minimal warmth, slightly engaging but largely formal or distant.
          - Three stars: The tone is moderately friendly, balancing formal and informal elements to somewhat engage the reader.
          - Four stars: The tone is very friendly, actively engaging the reader with warmth and lively expressions.
          - Five stars: The tone is exceptionally friendly and lively, creating a strong sense of connection and engagement with the reader.

          This rating should always be an integer between 1 and 5. Thus, the rating produced should be 1, 2, 3, 4, or 5, based on your assessment of how friendly and engaging the text is towards the read.

          Evaluation Template (Score Only):

          Friendliness:
          """,
"""Fluency measures the quality of individual sentences in the answer, and whether they are well-written and grammatically correct. Consider the quality of individual sentences when evaluating fluency. Given the question and answer, score the fluency of the answer between one to five stars using the following rating scale: 

          One star: the answer completely lacks fluency 

          Two stars: the answer mostly lacks fluency 

          Three stars: the answer is partially fluent 

          Four stars: the answer is mostly fluent 

          Five stars: the answer has perfect fluency 

          This rating value should always be an integer between 1 and 5. So the rating produced should be 1 or 2 or 3 or 4 or 5.


          Evaluation Template (Score Only):

          Fluency:
          """,
"""Human-likeness, as a metric, evaluates how closely the style and delivery of GPT-generated text resemble that of naturally written human text. If the text is indistinguishable from human writing in terms of natural flow, vocabulary use, and contextual accuracy, then the value of the Human-likeness metric should be high; otherwise, it should be low. Given the text generated by GPT and comparing it to typical human-written text, determine the value of the Human-likeness metric using the following rating scale:

          - One star: The text is entirely machine-like, with unnatural flow and numerous contextual errors.
          - Two stars: The text is mostly machine-like, with frequent unnatural expressions and noticeable errors.
          - Three stars: The text is somewhat human-like, showing signs of natural language use but with some mechanical characteristics.
          - Four stars: The text is mostly human-like, with very few mechanical traits and generally natural language use.
          - Five stars: The text is completely human-like, indistinguishable from something a human might write.

          This rating should always be an integer between 1 and 5. Thus, the rating produced should be 1, 2, 3, 4, or 5, based on the evaluation.
          """
]

### 3-2. 평가 시작 (평가에서는 각각의 새로운 채팅으로 하였음) -> memory 사용x

In [3]:
result

"안녕하세요, 여러분! 저희 '지하철 온다'의 마스코트 오지통이 인사드립니다. 오늘도 여러분의 출퇴근길이 순조로우셨기를 바라며, 최근 있었던 지하철 지연 사고에 대해 알려드리려고 해요.\n\n**지연 사고 개요:**\n우리 모두가 불편을 겪었던 그 사고! 바로 어제, 출근 시간대에 발생한 지하철 지연 사건이랍니다.\n\n**세부사항:**\n- **일시**: 2023년 10월 5일 오전 8시\n- **영향을 받은 노선**: 2호선 및 7호선\n- **지연 원인**: 신호 장애로 인해 발생했어요.\n\n이런 사고가 발생하면 정말 답답하죠? 하지만 이런 사고를 통해 우리는 더 안전하고 믿을 수 있는 서비스를 제공하기 위한 중요한 교훈을 얻게 되어요. 잠시의 불편함을 겪을 수는 있지만, 이를 통해 더 나은 서비스를 위한 발판을 마련하고 있다고 생각해 주세요.\n\n**지하철 온다 서비스 홍보:**\n'지하철 온다' 앱을 통해 실시간 업데이트를 받아보세요. 또한, 경로를 계획할 수 있는 사용자 친화적인 기능들을 많이 갖추고 있답니다. 여러분의 피드백은 우리 서비스 개선에 큰 도움이 됩니다. 자주 이용하시는 분들을 위한 로열티 프로그램도 잊지 마세요!\n\n**마무리 인사:**\n앞으로 더 밝고 매끄러운 날들이 기다리고 있다고 확신합니다. '지하철 온다'는 계속해서 여러분의 편안하고 신속한 여행을 위해 노력할 것입니다. 이번 지연 사건에 대한 여러분의 인내와 이해에 감사드리며, 앞으로 더욱 편리하고 지연 없는 여행을 기대해 주세요. 오늘도 안전하고 즐거운 하루 되세요!"

추후 코드 통합할때 source를 직접 가져올 예정

In [4]:
source="""전국철도노동조합(철도노조)의 준법투쟁으로 출근길 서울 지하철 1호선 이용객
이 불편을 겪고 있다.
28일 뉴스1 취재를 종합하면 1호선 지하철은 철도노조의 태업으로 지연 운행
하고 있다. 도심 지하철역은 출근시간대 이용객이 몰리며 큰 혼잡을 빚고 있다.
종각역으로 출근하는 20대 김모씨는 "평소보다 느리게 운행해 열차 안이 승객
들로 빽빽했다"며 "철도노조 태업 때문에 운행이 지연된다는 안내방송을 하면
서 급한 고객은 다른 교통수단을 이용하라는데 그러려면 몇 번이나 환승해야
"월요일부터 지각" 지하철 1호선 지연 운행에 시민 불
편…"노조 준법투쟁"
출근 시간 승객 큰 불편…지연 운행에 지각 우려
부산―수서 SRT 운행축소에 노조 24일부터 태업
해 불편하다"고 답답함을 호소했다. 
온라인에는 "1호선 연착으로 지각하게 생겼다"거나 "연착이 심해 역사 안에 발
디딜 곳이 없다" "개강 첫 날 비까지 오는데 불편하다" "퇴근길이 걱정된다"는
반응이 올라왔다. 
한국철도공사(코레일) 관계자는 "특별한 사건 사고가 있는 것은 아니기 때문에
철도노조의 준법투쟁으로 지연 운행되는 것 같다"면서 "정확한 지연 시간을 살
피고 있다"고 말했다. 
철도노조는 국토교통부가 사회적 논의나 공론화 없이 내달 1일부터 부산―수
서를 운행하는 SRT 고속열차의 운행을 11% 이상(일일 4100여석) 축소한 것에
반발해 지난 24일부터 준법투쟁(태업)을 하고 있다.
"""

In [34]:
eval_result=[]
#정확성- 기사(source)와 블로그 콘텐츠 모두 넣어야함
for i in eval[0]: 
    prompt = ChatPromptTemplate.from_messages([
    ("system", i),
    ("user", "{input}")
])
    chain = prompt | llm
    eval_result+=chain.invoke({"input": "blog content:"+result+"article, source:"+source})

#그외 평가방법은 기사(source)가 필요없음, input token 절약
for i in eval[1:]: 
    prompt = ChatPromptTemplate.from_messages([
    ("system", i),
    ("user", "{input}")
])
    chain = prompt | llm
    eval_result+=chain.invoke({"input": "blog content:"+result}) 

KeyboardInterrupt: 

In [59]:
eval_result

[('content',
  "Title: 지하철 지연 사고 및 철도노조 준법투쟁에 대한 이해와 '지하철 온다' 서비스 소개\n\n안녕하세요, 여러분! '지하철 온다'의 마스코트 오지통입니다. 최근 지하철 이용에 불편을 겪으신 모든 분께 진심으로 위로의 말씀을 전합니다. 오늘은 지하철 지연 사고와 철도노조의 준법투쟁에 대해 이야기하고자 합니다.\n\n### 최근 발생한 지하철 지연 사고\n2023년 10월 5일 오전 8시경, 2호선 및 7호선에서 신호 장애로 인한 지연이 발생했습니다. 이로 인해 많은 승객분들이 출근길에 큰 불편을 겪으셨습니다. 이런 사고는 답답하고 불편하게 느껴지지만, 안전과 서비스 품질 향상을 위한 중요한 교훈으로 삼고 있습니다.\n\n### 철도노조의 준법투쟁 배경\n한편, 철도노조는 부산-수서 SRT 운행 축소 결정에 반발하여 2023년 8월 24일부터 준법투쟁(태업)에 돌입했습니다. 이로 인해 1호선을 포함한 여러 노선의 지연 운행이 발생하고 있으며, 많은 시민들이 큰 불편을 겪고 있습니다.\n\n### '지하철 온다' 서비스 소개\n이런 상황에서도 '지하철 온다' 앱은 여러분께 도움을 드리고자 최선을 다하고 있습니다. 실시간 업데이트를 통해 지하철 운행 정보를 빠르게 제공하며, 경로 계획과 같은 사용자 친화적인 기능도 갖추고 있습니다. 여러분의 피드백은 우리 서비스 개선에 큰 도움이 됩니다, 그리고 로열티 프로그램도 잊지 마세요!\n\n### 마무리 인사\n'지하철 온다'는 앞으로도 여러분의 편안하고 신속한 여행을 위해 끊임없이 노력할 것입니다. 이번 지연 사건 및 철도노조의 준법투쟁에 대한 여러분의 인내와 이해에 감사드립니다. 앞으로 더욱 편리하고 지연 없는 여행을 기대해 주시기 바랍니다. 오늘도 안전하고 즐거운 하루 되시길 바랍니다!"),
 ('additional_kwargs', {}),
 ('response_metadata',
  {'token_usage': {'completion_tokens': 829,
    'prompt_t

In [1]:
import pandas as pd

In [11]:
pd.DataFrame({"link":[source_link[0]],"source":[source],"result":[result]})

link  \
0  https://www.news1.kr/articles/5152573   

                                              source  \
0  전국철도노동조합(철도노조)의 준법투쟁으로 출근길 서울 지하철 1호선 이용객\n이 불...   

                                              result  
0  안녕하세요, 여러분! 저희 '지하철 온다'의 마스코트 오지통이 인사드립니다. 오늘도...

비용문제때문에 코드로는 한개의 기사만 적용하였고 웹으로 대부분 실험 진행하였음